# Advent of Code - Day 3

In [1]:
import os
import json

## Load File Content First

In [2]:
FILE_PATH = "input.txt"

def open_file(file_path: str) -> list:
    """
    Open file
    The file is a txt file
    """
    with open(file_path, 'r') as file:
        content = [line.strip() for line in file]
    return content

FILE_CONTENT = open_file(FILE_PATH)
assert isinstance(FILE_CONTENT, list)
content_size = len(FILE_CONTENT)
print(f"Length of file content: {content_size}")

Length of file content: 6


## Strategy

1. We need a logic that can read each line, and filter it based on the pattern: `mul(x,y)`
2. This can be achieved using a regex
3. Then we can create a function that takes in that string and multiples the number.
4. Note that the length of each param is 1 to 3.

In [3]:
import re

def get_mul_v1(pattern: str) -> int:
    """
    Get the multiplication of the numbers in the text
    """
    # TODO: Handle spaces in the string because we should not consider those patterns
    if " " in pattern:
        return 0
    pattern = pattern.split("mul")[1].split("(")[1].split(")")[0].split(",")
    pattern = [int(i) for i in pattern]
    return pattern[0] * pattern[1]


def catch_mul_pattern_v1(text: str) -> int:
    """
    Catch the multiplication pattern
    """
    pattern: list = re.findall(r"mul\(\d+,\d+\)", text)
    return pattern


def get_sum_pattern_v1(content: str) -> int:
    result: list = []
    for string in content:
        pattern = catch_mul_pattern_v1(string)

        for p in pattern:
            result.append(get_mul_v1(p))

    return sum(result)

part_1_result = get_sum_pattern_v1(FILE_CONTENT)
print(f"Part 1 - The sum of the multiplication is: {part_1_result}")


Part 1 - The sum of the multiplication is: 190604937


### Part 1 is CORRECT!

### PART 2 - Enable `do()` and `don't()`

In [4]:
import re
from typing import List, Tuple

def catch_mul_pattern_v2(text: str):
    """
    Catch the multiplication pattern based on the specified conditions:
    1. Get all text from the start until the first `don't()`.
    2. Ignore the string until the first `do()`.
    3. Capture all strings between each `do()` and the next `don't()` or until the end if no `don't()` exists after a `do()`.
    """
    # Step 1: Capture everything before the first `don't()`
    before_dont_match = re.search(r"^(.*?)don't\(\)", text)
    if before_dont_match:
        before_dont = before_dont_match.group(1)
        # Remove the matched portion from the text for further processing
        text = text[before_dont_match.end():]
    else:
        before_dont = ""

    # Step 2: Capture all segments between `do()` and `don't()`
    do_dont_matches = re.findall(r"do\(\)(.*?)(?=don't\(\)|$)", text)

    # Concatenate all segments into a single string
    do_dont_matches = "".join(do_dont_matches)
    
    return before_dont, do_dont_matches

# Start iterating
def get_sum_pattern_v2(content: str) -> int:
    result: list = []
    for string in content:
        string_v2_p1, string_v2_p2 = catch_mul_pattern_v2(string)
        string_v2 = string_v2_p1 + string_v2_p2
        result.append(get_sum_pattern_v1([string_v2]))

    return sum(result)

part_2_result = get_sum_pattern_v2(FILE_CONTENT)
print(f"Part 2 - The sum of the multiplication is: {part_2_result}")

Part 2 - The sum of the multiplication is: 87020895


#### After multiple attempts at this, I am still getting `87020895` which AoC says is "TOO HIGH"

#### So, we need to change the approach

In [5]:
# Assume that the disabled instructions are continued to be disabled until the next `do()` instruction through the new line too.
CONCAT_FILE_CONTENT = " ".join(FILE_CONTENT)
part_2_result = get_sum_pattern_v2([CONCAT_FILE_CONTENT])
print(f"Part 2 - The sum of the multiplication is: {part_2_result}")

Part 2 - The sum of the multiplication is: 82857512


### `82857512` IS THE CORRECT ANSWER!